## 1. 各種設定

Zennを参照しながら下記３つのトークン、シークレットを取得し、設定してください。

1. LINE Bot のチャンネルアクセストークンを設定します
2. LINE Bot のチャンネルシークレットを設定します
3. ngrokのauthtokenを設定します

LINE API
https://developers.line.biz/ja/services/messaging-api/

ngrok
https://ngrok.com/

In [ ]:
# LINE Bot のチャンネルアクセストークン
LINEBOT_ACCESS_TOKEN = ''

# LINE Bot のチャンネルシークレット
LINEBOT_SECRET = ''

# ngrokのauthtoken
NGROK_AUTHTOKEN = ''

## 2. ngrok

ngrokを利用してこのノートブックを公開し、外からアクセスできる状態にします。

まずは必要なライブラリをインストールし、トークンを指定します。


In [ ]:
!pip install pyngrok
!ngrok authtoken $NGROK_AUTHTOKEN

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 5.3 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=28a672f6c9cb2076ea5fcc5e6fe0a06a433609b193c21b6151b3bb36056be864
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


実行します。

In [ ]:
from pyngrok import ngrok
from pyngrok.conf import PyngrokConfig

import os

os.system('kill -9 $(pgrep ngrok)')
public_url = ngrok.connect(addr='127.0.0.1:5000', pyngrok_config=PyngrokConfig(start_new_session=True))
print (public_url)

NgrokTunnel: "http://8361-35-227-112-27.ngrok.io" -> "http://127.0.0.1:5000"


↑
http://xxxxxxx.ngrok.io
の「http」を「https」に変更し、LINE Developers内のWebhook URLに入力して下さい。

## 3. アプリの準備

ライブラリをインストールします。

アプリで使用するライブラリをインストールします。↑トークン設定後に１回実行してください。

（ページを開きなおしたときも実行してください）

In [ ]:
!pip install flask==0.12.2
!pip install line-bot-sdk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 1.1 MB/s 
  Attempting uninstall: flask
    Found existing installation: Flask 1.1.4
    Uninstalling Flask-1.1.4:
      Successfully uninstalled Flask-1.1.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 3.5 MB/s 


## 4. メインプログラムの実行（ブラックジャック）

メインプログラムです。上記の準備が整ったら実行してください。

In [ ]:
from flask import Flask, request, abort

from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
)

from concurrent.futures import ProcessPoolExecutor
import random
import datetime
#import schedule
import time


app = Flask(__name__)

line_bot_api = LineBotApi(LINEBOT_ACCESS_TOKEN)
handler = WebhookHandler(LINEBOT_SECRET)

# ユーザーがLINE Botに送信した情報を受け取り、handlerに渡す
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

# Number of Matchs
# num_matchs = 10

# list of matchs. [player1_ID, player2_ID, match admin, last access day] should be append
matchs = []

# sp-cards. [name, probability, explain]
sp_cards = [
      ['Guard 1', 0.2, ': your bet will decrease 1'],
      ['Bet +1', 0.2, ': both bet will rise 1'],
      ['Draw Best Card', 0.1, ': brings the best card for you. turn will not end'],
      ['Change Goal to 25', 0.2, ': goal will be set to 25'],
      ['Change Goal to 17', 0.2, ': goal will be set to 17'],
      ['Exchange Last Card', 0.2, ': your last card open and opponent\'s last card open will be exchanged'],
      ['Return Card', 0.2, ': your last card will go back to the deck'],
      ['Remove Card', 0.2, ': opponent\'s your last card will go back to the deck'],
      ['Draw SP Card', 0.05, ': draw 2 SP Cards. turn will not end']
    ]
# Probability of drawing sp-card
sp_prob = 0.1

# [player1, player2, ...]
publiclist = []

# [roomnumber, player1]
waitinglist = []

class player:
  global sp_cards

  def __init__(self):
    self.stock = 3
    self.normal_cards = []
    self.sp_cards_list = [0] * len(sp_cards)

  def __del__(self):
    print('broken1')

  def life(self, n):
    self.stock-=n
    return self.stock

  def showcards():
    pass

  def addcard(self, k):
    self.normal_cards.append(k)

  def addspcard(self, k):
    self.sp_cards_list[k]+=1

  def usespcard(self, k):
    if self.sp_cards_list[k]==0:
      return 1
    else:
      self.sp_cards_list[k]-=1
      return 0

  def returnlastcard(self):
    if len(self.normal_cards)>1:
      k = self.normal_cards[len(self.normal_cards)-1]
      self.normal_cards.pop(len(self.normal_cards)-1)
      return k
    else:
      return None

  def resetnorm(self):
    self.normal_cards = []

  def getsum(self):
    sum = 0
    for i in range(len(self.normal_cards)):
      sum += self.normal_cards[i]
    return sum

  def getopponentcardinfo(self):
    mes='Normal : ?'
    sum = 0
    for i in range(len(self.normal_cards)-1):
      mes+=' %d' % (self.normal_cards[i+1])
    for i in range(len(self.sp_cards_list)):
      sum+=self.sp_cards_list[i]
    mes+='\nSP : %d cards' % (sum)
    return mes

  def getmycardinfo(self, n):
    mes='Normal :'
    sum = 0
    for i in range(len(self.normal_cards)):
      mes+=' %d' % (self.normal_cards[i])
    mes+='\nTotal : %d\nSP :' % (self.getsum())
    if n==0 or n==None:
      for i in range(len(self.sp_cards_list)):
        mes+=' %d' % (self.sp_cards_list[i])
    elif n==1:
      sum=0
      for i in range(len(self.sp_cards_list)):
        sum+=self.sp_cards_list[i]
      mes+=' %d cards' % (sum)
    return mes

class match_admin:
  global sp_prob

  def __init__(self):
    self.deck = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
    player1 = player()
    player2 = player()
    self.active_player = 0
    self.player_list = [player1, player2]
    self.game_num = 0
    self.status = 0
    self.startgame()
    self.draw_sp(0)
    self.draw_sp(1)
    self.sponfield=[[], []]

  def __del__(self):
    print('broken2')

  def startgame(self):
    self.deck = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
    self.player_list[0].resetnorm()
    self.player_list[1].resetnorm()
    for i in range(4):
      self.draw_norm()
    self.draw_sp(0)
    self.draw_sp(1)
    self.bet = [1, 1]
    self.goal = 21
    self.sponfield=[[], []]
    self.active_player = self.game_num % 2
    self.game_num+=1

  def gameresult(self):
    # return loser
    sum1 = self.player_list[0].getsum()
    sum2 = self.player_list[1].getsum()
    if sum1 <= self.goal and sum2 <= self.goal:
      if sum1>sum2:
        return 1
      elif sum1<sum2:
        return 0
      else:
        return 2
    elif sum1 > self.goal and sum2 > self.goal:
      if sum1<sum2:
        return 1
      elif sum1>sum2:
        return 0
      else:
        return 2
    elif sum1 > self.goal:
      return 0
    else:
      return 1

  def draw_norm(self):
    i = random.randrange(len(self.deck))
    k = self.deck[i]
    self.deck.pop(i)
    self.player_list[self.active_player].addcard(k)
    self.active_player=1-self.active_player

  def draw_sp(self, n):
    global sp_cards
    sum1=0
    sum2=0
    for i in range(len(sp_cards)):
      sum1+=sp_cards[i][1]
    t = random.random()
    for i in range(len(sp_cards)):
      sum2+=sp_cards[i][1]/sum1
      if t<sum2:
        self.player_list[n].addspcard(i)
        break

  def draw(self):
    self.status = 0
    # Max num of SP Cards drawed at once
    count = 3
    while random.random()<sp_prob and count>0:
      count-=1
      self.draw_sp(self.active_player)
    self.draw_norm()

  def check(self):
    self.status+=1
    self.active_player=1-self.active_player
    if self.status==2:
      # i is loser number
      i = self.gameresult()
      if not i==2:
        if self.bet[i]<0:
          self.bet[i]=0
        k = self.player_list[i].life(self.bet[i])
        if k<=0:
          return self.matchresult()
        return 1-i
      return 2
    return -1

  def getturn(self):
    return self.active_player

  def matchresult(self):
    if self.player_list[0].stock > self.player_list[1].stock:
      return 10
    elif self.player_list[0].stock < self.player_list[1].stock:
      return 11
    else:
      return 12

  def usesp(self, n):
    self.status = 0
    if self.player_list[self.active_player].sp_cards_list[n]<=0:
      return 1
    else:
      if self.spadmin(n)==2:
        return 2
      else:
        self.sponfield[self.active_player].append(n)
        self.player_list[self.active_player].usespcard(n)
        return 0

  # sp functions
  def spadmin(self, n):
    if n==0:
      return self.guardcard()
    elif n==1:
      return self.addbet()
    elif n==2:
      return self.bestcard()
    elif n==3:
      return self.goalcard(25)
    elif n==4:
      return self.goalcard(17)
    elif n==5:
      return self.exchangecard()
    elif n==6:
      return self.returncard()
    elif n==7:
      return self.removecard()

  # 6
  def returncard(self):
    if len(self.player_list[self.active_player].normal_cards)<2:
      return 2
    temp = self.player_list[self.active_player].returnlastcard()
    if not temp==None:
      self.deck.append(temp)
      return 0
    else:
      return 2

  # 0
  def guardcard(self):
    if self.bet[self.active_player]>0:
      self.bet[self.active_player]-=1
      return 0
    else:
      return 2

  # 1
  def addbet(self):
    self.bet[0]+=1
    self.bet[1]+=1
    return 0

  # 3, 4
  def goalcard(self, n):
    if self.goal==n:
      return 2
    else:
      self.goal = n
      return 0

  # 2
  def bestcard(self):
    if len(self.deck)<1:
      return 2
    total = self.player_list[self.active_player].getsum()
    i = self.goal - total
    if i>0:
      for k in range(len(self.deck)):
        if self.deck.count(i)>0:
          self.player_list[self.active_player].addcard(i)
          self.deck.remove(i)
          return 0
        else:
            i-=1
            if i<1:
              break
    if i<1:
      for k in range(len(self.deck)):
        if self.deck.count(k+1)>0:
          self.player_list[self.active_player].addcard(k+1)
          self.deck.remove(k+1)
          return 0
    return 2

  # 5
  def exchangecard(self):
    if len(self.player_list[self.active_player].normal_cards)<2 or len(self.player_list[1-self.active_player].normal_cards)<2:
      return 2
    temp = self.player_list[self.active_player].normal_cards[-1]
    self.player_list[self.active_player].normal_cards.pop(-1)
    self.player_list[1-self.active_player].normal_cards.append(temp)
    temp = self.player_list[1-self.active_player].normal_cards[-2]
    self.player_list[1-self.active_player].normal_cards.pop(-2)
    self.player_list[self.active_player].normal_cards.append(temp)
    return 0

  # 7
  def removecard(self):
    if len(self.player_list[1-self.active_player].normal_cards)<2:
      return 2
    temp = self.player_list[1-self.active_player].returnlastcard()
    if not temp==None:
      self.deck.append(temp)
      return 0
    else:
      return 2

  # 8
  def getspcard(self):
    self.draw_sp(self.active_player)
    self.draw_sp(self.active_player)
    return 0

def checklastlogin():
  global matchs
  global publiclist
  global waitinglist
  maxwaittime=1800 # seconds
  dt1 = datetime.datetime.now()
  print('check')
  for i in range(len(matchs)):
    if (dt1-matchs[i][3]).seconds> maxwaittime:
      line_bot_api.push_message(matchs[i][0], TextSendMessage(text='Time out'))
      line_bot_api.push_message(matchs[i][1], TextSendMessage(text='Time out'))
      matchs.pop(i)
      i-=1
  for i in range(len(publiclist)):
    if (dt1-publiclist[i][1]).seconds> maxwaittime:
      line_bot_api.push_message(publiclist[i][0], TextSendMessage(text='Time out'))
      publiclist.pop(i)
      i-=1
  for i in range(len(waitinglist)):
    if (dt1-waitinglist[i][2]).seconds> maxwaittime:
      line_bot_api.push_message(waitinglist[i][1], TextSendMessage(text='Time out'))
      waitinglist.pop(i)
      i-=1

def makepublicroom(id):
  global publiclist
  global waitinglist
  global matchs
  dt1 = datetime.datetime.now()

  for i in range(len(publiclist)):
    if id==publiclist[i][0]:
      publiclist.pop(i)
      break
  for i in range(len(waitinglist)):
    if id==waitinglist[i][1]:
      waitinglist.pop(i)
      break

  if len(publiclist)==0:
    publiclist.append([id, dt1])
    return -1
  else:
    newmatch = match_admin()
    matchs.append([publiclist[0][0], id, newmatch, dt1])
    return len(matchs)-1

def makeprivateroom(id):
  global publiclist
  global waitinglist
  roomnum = 10000
  roomnum+=random.randrange(90000)
  dt1 = datetime.datetime.now()
  k=0

  for i in range(len(publiclist)):
    if id==publiclist[i][0]:
      publiclist.pop(i)
      break

  while k<len(waitinglist):
    for i in range(len(waitinglist)):
      if waitinglist[i][1]==id:
        return waitinglist[i][0]
      if waitinglist[i][0]==roomnum:
        roomnum = 10000
        roomnum+=random.randrange(90000)
        k=0
        break
      k+=1
  waitinglist.append([roomnum, id, dt1])
  return roomnum

def joinroom(id, roomnum):
  global publiclist
  global waitinglist
  global matchs
  dt1 = datetime.datetime.now()

  for i in range(len(publiclist)):
    if id==publiclist[i][0]:
      publiclist.pop(i)
      break
  for i in range(len(waitinglist)):
    if id==waitinglist[i][1]:
      waitinglist.pop(i)
      break

  for i in range(len(waitinglist)):
    if waitinglist[i][0]==roomnum:

      for i in range(len(publiclist)):
        if id==publiclist[i][0]:
          publiclist.pop(i)
          break
      for i in range(len(waitinglist)):
        if id==waitinglist[i][1]:
          waitinglist.pop(i)
          break

      newmatch = match_admin()
      matchs.append([waitinglist[i][1], id, newmatch, dt1])
      return len(matchs)-1
  return -1

def findmatch(id):
  global matchs
  dt1 = datetime.datetime.now()
  for i in range(len(matchs)):
    if id == matchs[i][0]:
      matchs[i][3]=dt1
      return [i, matchs[i], 0]
    if id == matchs[i][1]:
      matchs[i][3]=dt1
      return [i, matchs[i], 1]
  return None

def showboard(match, playernum):
  mes = 'Goal : %d\n\n' % (match[2].goal)
  mes+= 'My Cards\n'
  mes+= match[2].player_list[playernum].getmycardinfo(0)
  mes+= '\nLife : %d' % (match[2].player_list[playernum].stock)
  mes+= '\nBet : %d' % (match[2].bet[playernum])
  mes+= '\n'
  mes+= '\nOpponent Cards\n'
  mes+= match[2].player_list[1-playernum].getopponentcardinfo()
  mes+= '\nLife : %d' % (match[2].player_list[1-playernum].stock)
  mes+= '\nBet : %d' % (match[2].bet[1-playernum])
  return mes

def showresultboard(match, playernum):
  mes = '----Result----\n\n'
  mes+= 'Goal : %d\n\n' % (match[2].goal)
  mes+= 'My Cards\n'
  mes+= match[2].player_list[playernum].getmycardinfo(0)
  mes+= '\nLife : %d' % (match[2].player_list[playernum].stock)
  mes+= '\n'
  mes+= '\nOpponent Cards\n'
  mes+= match[2].player_list[1-playernum].getmycardinfo(1)
  mes+= '\nLife : %d' % (match[2].player_list[1-playernum].stock)
  mes+= '\n--------------'
  # match[2].startgame()
  return mes

helpmes = '---- HELP ----\n\n\n'
helpmes+='- Commands\n\n'
helpmes+='whenever\n'
helpmes+='  '+'{: ^15s}'.format('help')+': to see this message\n'
helpmes+='  '+'{: ^15s}'.format('rule')+': to see the rules\n'
helpmes+='  '+'{: ^15s}'.format('bye')+': leave from match / resign matching\n'

helpmes+='\nwhile NOT in match\n'
helpmes+='  '+'{: ^15s}'.format('make room')+': make private room\n'
helpmes+='  '+'{: ^15s}'.format('random')+': join random match\n'
helpmes+='  '+'{: ^15s}'.format('10000-99999')+': if the room number exist, then join\n'

helpmes+='\nwhile NOT in match\n'
helpmes+='  '+'{:^15s}'.format('show')+': to see the situation\n'
helpmes+='  '+'{:^15s}'.format('draw')+': to draw card. turn will end\n'
helpmes+='  '+'{:^15s}'.format('check')+': to stay. if both checked in sequence, win will be decided\n'
helpmes+='  '+'{:^15s}'.format('0-4')+': use the special card of the number\n'

helpmes+='\n\n- SP Cards\n\n'


rulemes='This is a blackjack-like game.\n\nThere are 11 cards numbered 1-11 in the deck. Make your sum of the card close to goal. If BOTH player\'s sum where not over the goal or BOTH player\'s sum where over the goal, the closest player will win the game. If only one of the player over the goal, player who did not over the goal will win the game. If the opponent\'s life reaches 0, you will win the match. \nThere are SP cards.\n\nSend \'help\' to see the SP cards and commands to play'

# ユーザーがBotにテキストを送信した時の処理。
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    global waitinglist
    global publiclist
    global matchs
    global sp_cards
    global sp_prob
    global helpmes
    global rulemes

    # メッセージを格納する配列。3通まで同時に送ることが可能
    message_to_user = []

    checklastlogin()

    if event.message.text == 'help':
      #message_to_user.append(TextSendMessage(text='making text...'))
      for i in range(len(sp_cards)):
        helpmes+='  '+'{:^15d}'.format(i)+sp_cards[i][2]+'\n'
      message_to_user.append(TextSendMessage(text=helpmes))
    elif event.message.text == 'rule':
      message_to_user.append(TextSendMessage(text=rulemes))
    else:
      '''
      profile = line_bot_api.get_profile(event.source.user_id)
      id = profile.user_id
      print(id)
      '''
      id = event.source.user_id
      #print(id)
      inmatch = findmatch(id)

      if inmatch==None:
        # message_to_user.append(TextSendMessage(text='not in match'))
        # make private room
        if event.message.text == 'make room':
          k = makeprivateroom(id)
          message_to_user.append(TextSendMessage(text='Send the room number below to your opponent'))
          message_to_user.append(TextSendMessage(text='%d' % (k)))
          message_to_user.append(TextSendMessage(text='Wait for your opponent to enter...'))

        # make public room
        elif event.message.text == 'random':
          s=makepublicroom(id)
          if s==-1:
            message_to_user.append(TextSendMessage(text='Wait for Opponent...'))
          else:
            line_bot_api.push_message(matchs[s][0], TextSendMessage(text=showboard(matchs[s], 0)))
            line_bot_api.push_message(matchs[s][0], TextSendMessage(text='Opponent entered\nYour turn\nEnter action'))
            message_to_user.append(TextSendMessage(text=showboard(matchs[s], 1)))
            message_to_user.append(TextSendMessage(text='Match Found\nOpponent Turn\nWait for your turn...'))

        elif event.message.text.isnumeric() and 10000 <= int(event.message.text) and int(event.message.text)<100000:
          s = joinroom(id, int(event.message.text))
          if not s==-1:
            message_to_user.append(TextSendMessage(text=showboard(matchs[s], 1)))
            message_to_user.append(TextSendMessage(text='entered room %d\nOpponent Turn\nWait for your turn...' % (int(event.message.text))))
            line_bot_api.push_message(matchs[s][0], TextSendMessage(text=showboard(matchs[s], 0)))
            line_bot_api.push_message(matchs[s][0], TextSendMessage(text='Opponent entered\nYour turn\nEnter action'))

        elif event.message.text == 'bye':
          message_to_user.append(TextSendMessage(text='Thank You'))
          for i in range(len(waitinglist)):
            if id==waitinglist[i][1]:
              waitinglist.pop(i)
              break
          for i in range(len(publiclist)):
            if id==publiclist[i][0]:
              publiclist.pop(i)
              break
      else:
        # message_to_user.append(TextSendMessage(text='in match'))
        if event.message.text == 'make room' or event.message.text == 'random match':
          message_to_user.append(TextSendMessage(text='already in room \nsend leave to get in new room'))

        elif event.message.text == 'bye':
          # send opponet win message
          message_to_user.append(TextSendMessage(text='You LOSE\nThank You For Playing'))
          line_bot_api.push_message(inmatch[1][1-inmatch[2]], TextSendMessage(text='Opponent left\nYour Win\nThank You For Playing'))
          matchs.pop(inmatch[0])

        elif event.message.text == 'show':
          message_to_user.append(TextSendMessage(text=showboard(inmatch[1], inmatch[2])))

        elif not inmatch[2]==inmatch[1][2].getturn():
          message_to_user.append(TextSendMessage(text='Not Your Turn\nWait for your turn...'))

        elif inmatch[2]==inmatch[1][2].getturn():
          if event.message.text.isnumeric() and 0 <= int(event.message.text) < len(sp_cards):
            k = inmatch[1][2].usesp(int(event.message.text))
            if k == 0:
              message_to_user.append(TextSendMessage(text='used sp%d:%s' % (int(event.message.text), sp_cards[int(event.message.text)][0])))
              message_to_user.append(TextSendMessage(text=showboard(inmatch[1], inmatch[2])))
              message_to_user.append(TextSendMessage(text='Enter action'))
              line_bot_api.push_message(inmatch[1][1-inmatch[2]], TextSendMessage(text='Opponent used sp%d:%s\nWait for your turn...' % (int(event.message.text), sp_cards[int(event.message.text)][0])))
              #line_bot_api.push_message(inmatch[1][1-inmatch[2]], TextSendMessage(text=showboard(inmatch[1], inmatch[2])))
            elif k==1:
              message_to_user.append(TextSendMessage(text='Not having sp%d' % (int(event.message.text))))
            elif k==2:
              message_to_user.append(TextSendMessage(text='Cannot use the sp card'))
          elif event.message.text == 'draw':
            inmatch[1][2].draw()
            message_to_user.append(TextSendMessage(text=showboard(inmatch[1], inmatch[2])))
            message_to_user.append(TextSendMessage(text='Turn end\nWait for your turn...'))
            line_bot_api.push_message(inmatch[1][1-inmatch[2]], TextSendMessage(text=showboard(inmatch[1], 1-inmatch[2])))
            line_bot_api.push_message(inmatch[1][1-inmatch[2]], TextSendMessage(text='Opponent drawed card\nYour Turn\nEnter action'))
          elif event.message.text == 'check':
            k = inmatch[1][2].check()
            if not k ==-1:
              message_to_user.append(TextSendMessage(text=showresultboard(inmatch[1], inmatch[2])))
              line_bot_api.push_message(inmatch[1][1-inmatch[2]], TextSendMessage(text=showresultboard(inmatch[1], 1-inmatch[2])))
              inmatch[1][2].startgame()
              if k%10==2:
                turnmes = ['Draw game\nYoue turn\nEnter action', 'Draw game\nOpponent turn\nWait for your turn...']
                if inmatch[2]==inmatch[1][2].getturn():
                  message_to_user.append(TextSendMessage(text=showboard(inmatch[1], inmatch[2])))
                  message_to_user.append(TextSendMessage(text=turnmes[0]))
                  line_bot_api.push_message(inmatch[1][1-inmatch[2]], TextSendMessage(text=showboard(inmatch[1], 1-inmatch[2])))
                  line_bot_api.push_message(inmatch[1][1-inmatch[2]], TextSendMessage(text=turnmes[1]))
                else:
                  message_to_user.append(TextSendMessage(text=showboard(inmatch[1], inmatch[2])))
                  message_to_user.append(TextSendMessage(text=turnmes[1]))
                  line_bot_api.push_message(inmatch[1][1-inmatch[2]], TextSendMessage(text=showboard(inmatch[1], 1-inmatch[2])))
                  line_bot_api.push_message(inmatch[1][1-inmatch[2]], TextSendMessage(text=turnmes[0]))
              else:
                if k>9:
                  messa = ['You Win the match\nThank You For Playing', 'You Lose the match\nThank You For Playing']
                  if k%10==inmatch[2]:
                    message_to_user.append(TextSendMessage(text=messa[0]))
                    line_bot_api.push_message(inmatch[1][1-inmatch[2]], TextSendMessage(text=messa[1]))
                  elif k%10==1-inmatch[2]:
                    message_to_user.append(TextSendMessage(text=messa[1]))
                    line_bot_api.push_message(inmatch[1][1-inmatch[2]], TextSendMessage(text=messa[0]))
                  matchs.pop(inmatch[0])
                else:
                  messa = ['You Win the game\n', 'You Lose the game\n']
                  turnmes = ['Your turn\nEnter action', 'Opponent turn\nWait for your turn...']
                  if k%10==inmatch[2]:
                    if inmatch[2]==inmatch[1][2].getturn():
                      messa[0]+=turnmes[0]
                      messa[1]+=turnmes[1]
                    else:
                      messa[0]+=turnmes[1]
                      messa[1]+=turnmes[0]
                    message_to_user.append(TextSendMessage(text=showboard(inmatch[1], inmatch[2])))
                    message_to_user.append(TextSendMessage(text=messa[0]))
                    line_bot_api.push_message(inmatch[1][1-inmatch[2]], TextSendMessage(text=showboard(inmatch[1], 1-inmatch[2])))
                    line_bot_api.push_message(inmatch[1][1-inmatch[2]], TextSendMessage(text=messa[1]))
                  elif k%10==1-inmatch[2]:
                    if not inmatch[2]==inmatch[1][2].getturn():
                      messa[0]+=turnmes[0]
                      messa[1]+=turnmes[1]
                    else:
                      messa[0]+=turnmes[1]
                      messa[1]+=turnmes[0]
                    message_to_user.append(TextSendMessage(text=showboard(inmatch[1], inmatch[2])))
                    message_to_user.append(TextSendMessage(text=messa[1]))
                    line_bot_api.push_message(inmatch[1][1-inmatch[2]], TextSendMessage(text=showboard(inmatch[1], 1-inmatch[2])))
                    line_bot_api.push_message(inmatch[1][1-inmatch[2]], TextSendMessage(text=messa[0]))
            else:
              message_to_user.append(TextSendMessage(text='Turn end\nWait for your turn...'))
              #message_to_user.append(TextSendMessage(text='Wait'))
              line_bot_api.push_message(inmatch[1][1-inmatch[2]], TextSendMessage(text=showboard(inmatch[1], 1-inmatch[2])))
              line_bot_api.push_message(inmatch[1][1-inmatch[2]], TextSendMessage(text='Opponent checked\nYour Turn\nEnter action'))

    # ユーザーに送信するメッセージが存在しない場合
    if len(message_to_user) == 0:
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text='invalid message'),
        )
    # メッセージが存在する場合は送信
    #'''
    else:
        line_bot_api.reply_message(
            event.reply_token,
            message_to_user,
        )
    #'''


if __name__ == "__main__":
    app.run()

INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


check


INFO:werkzeug:127.0.0.1 - - [27/Aug/2022 11:13:30] "POST / HTTP/1.1" 200 -


check


INFO:werkzeug:127.0.0.1 - - [27/Aug/2022 11:14:02] "POST / HTTP/1.1" 200 -


...

broken2
broken1
broken1
check


INFO:werkzeug:127.0.0.1 - - [27/Aug/2022 11:17:49] "POST / HTTP/1.1" 200 -


check


INFO:werkzeug:127.0.0.1 - - [27/Aug/2022 11:18:54] "POST / HTTP/1.1" 200 -


broken2
broken1
broken1


### 4.1 実行

### 概要

* 人vs人で「21」が遊べるBotです。CPUは搭載していません。
* 1人でも動作確認はできないです。
* ruleと送ることでルールが見れます。
* helpと送ることで有効なコマンドやSPカードの概要が見れます。
* 友達登録し、お互いrandomと送るか、一方がmake roomと送り返事された番号を相手が送ることで遊べます。

### 遊び方

* 先にrandomと送った方、またはmake roomと送った方が先攻となります。
* draw、check、またはSPカードの番号を送ることで操作できます。他のコマンドはhelpを送ると見れます。
* drawまたはcheckを送ると相手のターンになります。
* 両者checkのみを行ったターンが連続すると勝敗が決まります。

### 他

コードを変更したい場合はメインプログラムを停止し、変更後実行してください。ngrokの再起動は必要ありません。

ただし、時間経過によりGoogle Colabがサスペンドした場合はngrokも再起動が必要です。(その場合はLINE DevelopersにおいてWebhook URLを変更する必要があります。)
